# how to generate files

### - Download markdown of the person you want to add from Roam and add it to covert_files folder

### - run this and fix file if needed

In [34]:
import re
def clean(text):
  text = " ".join(text.split())
  text = text.replace("[[", "")
  text = text.replace("]]", "")
  text = text.replace("\n", "")
  text = text.replace("*", "")
  return text

def clean_dict(dictionary):
  d = {}
  for k, v in dictionary.items():
    if v != {}:
      d[k] = {}
    if  isinstance(v, list):
        d[k] = v
    else:
      for k2, v2 in v.items():
        if v2:
            d[k][k2] = v2
  return d

In [35]:
def read_lines(filename):
    with open(f"convert_files/{filename}", 'r') as f:
        lines = f.readlines()
    lines = [clean(line) for line in lines if clean(line)!='']
    return lines

In [36]:
def extract_person_dict(lines):
  person = {}
  section="other"
  for i, line in enumerate(lines):
      if "###" in line:
          section = line.replace("- ### ","")
          if section == "Vida" or section == "Documentos":
            person[section] = []
          else:
            person[section] = {}
          pass
      elif section == "Vida":
        person[section].append(clean(line).replace("- ", ""))
      elif section == "Documentos":
        person[section].append(line)
      elif ":" in line:
        vals = line.split(":")
        # print(len(vals))
        if ("Morte" == vals[0] and (clean(vals[1]) == 'DATA, LOCAL')) or ("Sepultamento" == vals[0] and (clean(vals[1]) == 'Cemitério de')):
          person[section][vals[0]] = ''
        elif "Filhos" in vals[0]:
          filhos = []
          for filho in lines[i+1:]:
            if filho[0]=="-":
              filhos.append(filho.replace("- ", ""))
            else:
              break
          person[section]["Filhos"] = filhos
        else:
          person[section][vals[0]] = clean(":".join(vals[1:]))

  person = clean_dict(person)
  # display(person)
  return person

In [37]:
from datetime import date
def create_header(name, dates):
  timest = dates[0] if len(dates)>0 else "????"
  timeend = dates[1] if len(dates)>1 else "????"
  today = date.today().strftime("%d/%m/%Y")
  s = f"---\ntitle: \'{name} ({timest}-{timeend})\'\ndata: {today}\nnome: {name}\nlayout: pessoa\n---\n\n"
  return s

def get_personal_data(personal_data):
  lines = []
  for k, v in personal_data.items():
    if k =="Pais":
      text = v.split(" e ")
      line = f'**{k}:** [{text[0]}](){{:class="underconstruction"}} e [{text[1]}](){{:class="underconstruction"}}<br/>'
      lines.append(line)
    elif k =="Casamento":
      text = v.split(",")
      line = f'**{k}:** [{text[0]}](){{:class="underconstruction"}}, {",".join(text[1:])}<br/>'
      lines.append(line)
    elif k =="Filhos":
      line = f"**{k}:**<br/>"
      lines.append(line)
      # print(v)
      for i, filho in enumerate(v):
        line = f'&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;&nbsp;{i+1}. [{filho}](){{:class="underconstruction"}}<br/>'
        lines.append(line)
    else:
      line = f"**{k}:** {v}<br/>"
      lines.append(line)
  return "\n".join(lines)

def get_life(vida):
  nl = '\n' 
  paragraphs = "\n\n".join(vida)
  text = f"{nl}{nl}## Vida:{nl}{nl}{paragraphs}{nl}"
  return text

def get_refs(refs):
  print(refs)
  nl = '\n' 
  paragraphs = "\n\n".join(refs)

  text = f"{nl}{nl}## Referências:{nl}{nl}{paragraphs}{nl}"
  return text

In [38]:
from os import listdir
from os.path import isfile, join
onlyfiles = [f for f in listdir("convert_files") if (isfile(join("convert_files", f)) and f !='.DS_Store')]
onlyfiles

['Christiano Frederico Scherer (1868-1945).md']

In [39]:
def get_time_name(filename):
  filename = filename.replace(".md", '')
  times = re.findall(r'\d+', filename)
  name = " ".join(re.sub(r'\([^)]*\)', '', filename).split())
  return times, name


In [40]:
def file_name_set(name, time):
  name_aux = "".join(name.split())
  date_aux = time[0] if len(time)>0 else ""
  return f"{name_aux}{date_aux}.md"

def save_file(name, time, person):
  f = open(f"converted/{file_name_set(name, time)}", "w")
  f.write(create_header(name, time))
  f.write(get_personal_data(person["Dados Pessoais"]))
  f.write(get_life(person["Vida"]))
  f.write(get_refs(person["Documentos"]))
  f.close()

In [41]:
for filename in onlyfiles:
  times, name = get_time_name(filename)
  lines = read_lines(filename)
  person = extract_person_dict(lines)
  save_file(name, times, person)


['Certidão de Nascimento: ', '[5] Registro de Batismo:  Batismos 1865-1885 (49)', '- https://drive.google.com/file/d/1BPonIaAnEOaNHa4OduUpdrHPXJ9slrmd/view?usp=sharing', '[3] Certidão de Casamento: https://www.familysearch.org/ark:/61903/3:1:3QS7-89L5-3B1S?i=1872&cat=58014 Christiano Frederico Scherer, Filo de Pedro Scherer e Elisabetha Schneider cc Carlotta Elisa Massmann Carlos Massmann e Elisa Laut', '[4] Certidão de Óbito:  https://www.familysearch.org/ark:/61903/3:1:3QS7-89GW-9W99?i=158&cat=58014', 'Outros:', '- [1] Certidão de óbito do pai Peter Scherer (1841-1917)', '- [2] Certidão de batismo da filha Malvina Scherer (1891-1962)', '- [6] Túmulo no Cemitério de Olarias -22 de abril de 1868 - 05 de fevereiro de 1945-https://drive.google.com/file/d/1UX1gwModVnmI8UZzjo8mvGrSbtddQU6Q/view?usp=sharing', 'Agradecimento:']


In [42]:
# FOLHA = "6"
# NUM = "9"
# LINK = "https://drive.google.com/file/d/11alIGA7qSWr9gIa5GBm4zfa3JhQ1NJi1/view?usp=sharing"
# DATA = "11 janeiro 2025"
# print(F"Registro de Casamento da Paróquia Evangélica de Conventos, Livro 1914-1935 Folha {FOLHA} Num. {NUM}. Acervo Pessoal. Disponível em: [{LINK}]({LINK}). Acesso em: {DATA}.")

In [47]:
date.today().strftime("%d %B/%Y")

'02 February/2025'

In [72]:
# from babel.dates import dateformat
from babel.dates import format_date, format_datetime, format_time

format_date(date.today(), format="long", locale="pt_BR")
format_date(date.today(), "dd 'de' MMMM 'de' YYYY", locale="pt_BR")
# "EEE, MMM d, ''yy

'02 de fevereiro de 2025'

In [73]:
from datetime import date

DATE = format_date(date.today(), "dd 'de' MMMM 'de' YYYY", locale="pt_BR")

def format_reg_civil_reference(doc_details):
    matricula = f' Número de Matrícula: {doc_details["matricula"]}.' if  doc_details["matricula"] else ''
    link = f' Disponível em: [{doc_details["link"]}]({doc_details["link"]}). Acesso em: {DATE}.' if  doc_details["link"] else ' Entrar em contato.'
    is_acervo = f' Acervo Pessoal.' if {doc_details["is_acervo"]} else ''
    return f'''[XX] Registro de {doc_details["type"]}. Registro Civíl de Pessoas Naturais - {doc_details["place"]}. Ano {doc_details["year"]}. Livro {doc_details["book"]}, Folha {doc_details["page"]}, Num. {doc_details["number"]}.{matricula}{is_acervo}{link}'''


In [74]:
# doc_details = {
#     "type": "Nascimento", # "Óbito" | Casamento
#     "place": "Lajeado, Rio Grande do Sul, Brasil",
#     "year": "2023",
#     "book": "A-27",
#     "page": "33v",
#     "number": "2341",
#     "matricula": None,
#     "link": None,
#     "is_acervo": True
# }

#  Livro B-21, Folha 150, Número 2633. Disponível em: https://drive.google.com/file/d/16-Zfv5ry6IV3AgX9WjZcoFfdtaNuLSwX/view?usp=sharing
doc_details = {
    "type": "Casamento", # Nascimento | "Óbito" | Casamento
    "place": "Lajeado, Rio Grande do Sul, Brasil",
    "year": "2023",
    "book": "B-21",
    "page": "150",
    "number": "2633",
    "matricula": None,
    "link": "https://drive.google.com/file/d/16-Zfv5ry6IV3AgX9WjZcoFfdtaNuLSwX/view?usp=sharing",
    "is_acervo": True
}
format_reg_civil_reference(doc_details)

'[XX] Registro de Casamento. Registro Civíl de Pessoas Naturais - Lajeado, Rio Grande do Sul, Brasil. Ano 2023. Livro B-21, Folha 150, Num. 2633. Acervo Pessoal. Disponível em: [https://drive.google.com/file/d/16-Zfv5ry6IV3AgX9WjZcoFfdtaNuLSwX/view?usp=sharing](https://drive.google.com/file/d/16-Zfv5ry6IV3AgX9WjZcoFfdtaNuLSwX/view?usp=sharing). Acesso em: 02 de fevereiro de 2025.'

In [ ]:
https://drive.google.com/file/d/10ujt7mkUy5VsMq8O-A4QlqSC7ftaK40K/view?usp=sharing